In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import kss
import re
import networkx as nx
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Mecab 객체 생성
mecab = Mecab()

In [6]:
# df 불러오기
df = pd.read_csv("./test.csv", header=None)
df.columns = ['article']

,article
0,영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구. © 로이터 뉴스1...
1,크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼로...
2,제주시는 최근 “신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다”고 했다....
3,5월엔 43.1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업 구...
4,서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생산...
...,...
6907,통계청 ‘5월 산업활동 동향’ 산업생산 5개월 연속 내리막 제조업 재고율도 외환위기...
6908,오늘부터 규제지역에서 주택담보대출을 받아 집을 사려면 6달 안에 새로운 집에 전입해...
6909,통계청 5월 산업동향 소비만 회복세 생산 1.2%·투자 5.9%·건설 4.3% 감소...
6910,농심은 최근 국내 e스포츠팀 팀 다이나믹스 에 대한 인수 협약을 맺고 e스포츠 분야...


In [7]:
df = df.iloc[0:5]

,article
0,영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구. © 로이터 뉴스1...
1,크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼로...
2,제주시는 최근 “신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다”고 했다....
3,5월엔 43.1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업 구...
4,서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생산...


In [8]:
# 중복 제거
df = df.drop_duplicates('article')
df.reset_index(inplace=True,drop=True)

In [14]:
#stopwords list화
temp = pd.read_csv("./stop_words.csv",header = None, usecols = [0], names = ['stopwords'])                                                
stopwords = []
for i in range(temp.shape[0]):
    stopwords.append(temp['stopwords'][i])

In [18]:
result = []
for i in range(len(df)):
    # 전처리
    removed = re.sub('[-=+,#/\?©:^$©@*\"※~&.·%｜!』\\‘◇”|\(\)\[\]\<\>`\'’“…》]', '', df['article'][i])
    
    # 토큰화
    tokenized = mecab.morphs(removed)
    for j in range(len(tokenized)-1,-1,-1):
        # stopwords 제거
        if tokenized[j] in stopwords:
            tokenized.remove(tokenized[j])
    result.append(tokenized)
df['token'] = result

In [19]:
df

,article,token
0,영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구. © 로이터 뉴스1...,"[영국, 옥스퍼드, 대, 신종, 코로나, 바이러스, 감염증, 코로나, 19, 백신,..."
1,크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼로...,"[크리스탈, 지노, 믹스, 수술, 힘든, 진행, 성, 췌장암, 환자, 발티노스, 타..."
2,제주시는 최근 “신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다”고 했다....,"[제주, 시, 는, 최근, 신산, 공원, 내, 저류지, 개방, 해, 반려, 동물, ..."
3,5월엔 43.1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업 구...,"[5, 엔, 431, 서울경제, 미국, 공급, 관리, 협회, ISM, 는, 1, 현..."
4,서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생산...,"[서울경제, 극자외선, 용, EUV, 포토레지스트, 생산, 는, 일본, 기업, TO..."


In [28]:
# Word2Vec 모델링
# news_tokens = df['token'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8
model = Word2Vec.load('word2vec.model')

# model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [40]:
#벡터화 작업
# input : pd.series
def vectorize(tokens):
    result = []
    for token in tokens:
        vectors = []
        for sentence in token:
            init_v = np.array([0.0]*v_dimension)
            count = 0
            
            ############################# 수정필요
            for word in sentence:
                word_vectors = model.wv
                if word in word_vectors.vocab:
                    count +=1
                    v = model.wv[word]
                    init_v = init_v + v
            init_v = init_v / count    
            vectors.append(init_v)
        result.append(vectors)
    
    frame = {'article' : df['article'], 'token': df['token'], 'vector': result }
    result = pd.DataFrame(frame) 
    
    return result

In [41]:
df = vectorize(df['token'])

영
국
옥
스
퍼
드
대
신
종
코
로
나
바
이
러
스
감
염
증
코
로
나
1
9
백
신
연
구
로
이
터
뉴
스
1
서
울
뉴
스
1
한
상
희
기
자
전
세
계
신
종
코
로
나
바
이
러
스
감
염
증
코
로
나
1
9
백
신
연
구
중
중
가
장
앞
선
평
가
받
는
영
국
옥
스
퍼
드
대
연
구
진
이
최
종
임
상
3
상
시
험
긍
정
적
인
결
과
얻
었
다
고
1
현
지
밝
혔
다
로
이
터
통
신
따
르
면
사
라
길
버
트
옥
스
퍼
드
대
의
대
교
수
는
이
날
의
회
출
석
코
로
나
1
9
백
신
임
상
3
상
진
행
상
황
대
한
질
문
백
신
후
보
물
질
적
절
한
종
류
면
역
반
응
보
였
다
고
답
했
다
옥
스
퍼
드
대
는
영
국
제
약
사
아
스
트
라
카
인
체
해
없
는
바
이
러
스
코
로
나
바
이
러
스
유
전
자
끼
워
넣
은
유
전
자
재
조
합
백
신
A
Z
D
1
2
2
2
개
발
이
달
8
0
0
0
명
임
상
시
험
자
원
자
대
상
3
상
시
험
고
있
다
앞
서
세
계
보
건
기
구
W
H
O
는
세
계
백
신
연
구
중
옥
스
퍼
드
아
스
트
라
제
네
카
가
가
장
앞
서
있
다
고
평
가
기
도
했
다
길
버
트
교
수
는
9
말
1
0
백
신
1
0
0
만
회
분
생
산
계
획
는
백
신
준
비
될
지
는
시
험
결
과
달
려
있
다
며
지
금
시
점
백
신
개
발
시
간
표
제
시
할
순
없
다
고
신
중
한
태
도
보
였
다
크
리
스
탈
지
노
믹
스
수
술
힘
든
진
행
성
췌
장
암
환
자
발
티
노
스
타
트
젬
시
타
빈
얼
로
티
닙
3
제
병
용
요
법
1
2
상
임
상
시
험
결
과
온
라
인
진
행
한
유
럽
종
양
학
회
소
화
기
암
국
제
회
의
발
표
했
임
상
시
험
은
수
술
불
가
능
한
진
행
성
또
는
전
이
성
췌
장
암
환
자
3
4
명
대
상
진
행
했
크
리
스
탈
지
노
믹
스
분
자
표
적
항
암
제
후
보
물
질
인
발
티
노
스
타


In [124]:
#코사인유사도 매트릭스
def similarity_matrix(vectors):
    sim_mat = np.zeros([len(vectors), len(vectors)])
    for i in tqdm(range(len(vectors))):
        for j in range(len(vectors)):
            sim_mat[i][j] = cosine_similarity(vectors[i].reshape(1, v_dimension), vectors[j].reshape(1, v_dimension))[0,0]
    return sim_mat


In [125]:
df['SimMatrix'] = df['vector'].apply(similarity_matrix)

100%|██████████| 129/129 [00:05<00:00, 24.34it/s]


In [126]:
df

,article,token,vector,SimMatrix
0,영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구. © 로이터 뉴스1...,"[영국, 옥스퍼드, 대, 신종, 코로나, 바이러스, 감염증, 코로나, 19, 백신,...","[[0.06985935149714351, -0.025657336227595806, ...","[[0.9999999999999998, -0.1314286774940165, 0.2..."
1,크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼로...,"[크리스탈, 지노, 믹스, 수술, 힘든, 진행, 성, 췌장암, 환자, 발티노스, 타...","[[-0.08433880284428596, -0.8906333353370428, -...","[[0.9999999999999999, -0.09545740665889839, 0...."
2,제주시는 최근 “신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다”고 했다....,"[제주, 시, 는, 최근, 신산, 공원, 내, 저류지, 개방, 해, 반려, 동물, ...","[[0.27231060713529587, 0.7067987825721502, 0.6...","[[0.9999999999999999, 0.4185591005579885, -0.0..."
3,5월엔 43.1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업 구...,"[5, 엔, 431, 서울경제, 미국, 공급, 관리, 협회, ISM, 는, 1, 현...","[[1.2809516191482544, -0.4342169165611267, 2.3...","[[1.0000000000000004, 0.12217151691247938, 0.7..."
4,서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생산...,"[서울경제, 극자외선, 용, EUV, 포토레지스트, 생산, 는, 일본, 기업, TO...","[[0.06795791210606694, -0.020151667529717088, ...","[[0.9999999999999999, 0.3334628596227865, -0.0..."


In [107]:
#코사인 유사도를 활용해서 각 문장마다 점수 매기기
#pagerank 사용
def calculate_score(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

--------- 이거 이렇게 하면 안될듯... word 하나당 벡터 300개씩 해버리면 계산량이 개지옥임..... 아무래도 keyword extraction은 tf-idf 써야할듯 ㅠㅠ------

참고: https://lovit.github.io/nlp/2019/04/30/textrank/